In [147]:
import requests
from bs4 import BeautifulSoup
import datetime
import os
import pymysql
from tqdm import tqdm
import time

In [140]:
ip_address = os.environ['AWS_IP']
PASSWORD = os.environ['SQL_PWD_AWS']

con = pymysql.connect(
    user='edward', 
    passwd=PASSWORD,
    host=ip_address, 
    db='community', 
    charset='utf8'
)

In [111]:
BASE_URL= 'https://www.fmkorea.com'

In [112]:
def get_urls(date):
    # %Y-%m-%d
    sql = f"select * from urls_fmk where date = '{date}'"
    with con.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
    return result

In [113]:
def is_dup_content(article_id):
    sql = f"select * from fmk_content where id = {article_id}"
    with  con.cursor() as cursor:
        cursor.execute(sql)
#     return bool(cursor.fetchall())
    return cursor.fetchall()

In [114]:
def commit_content(values):
    sql = f'insert into fmk_content(id, title, content, date) values {values}'
    with  con.cursor() as cursor:
        cursor.execute(sql)
        con.commit()

In [115]:
def commit_comment(values):
    sql = f'insert into fmk_comment(vote, content, article_id) values {values}'
    with  con.cursor() as cursor:
        cursor.execute(sql)
        con.commit()

In [149]:
urls = get_urls('2021-01-01')

In [158]:
headers = {'User-Agent': 'Mozilla/5.0'}

In [159]:
for u in tqdm(urls):
    article_id, url_tail, date = u
    if is_dup_content(article_id):
        continue
    url = BASE_URL + url_tail
    result = requests.get(url, headers = headers)
    soup = BeautifulSoup(result.text, 'lxml')

    title_bar = soup.find('div', {'class': 'top_area'})
    title = title_bar.find('h1').text
    content = '\n'.join([x.text for x in soup.find('article').find_all('p') if x.text])

    values = (article_id, title, content ,date.strftime('%Y-%m-%d'))

    commit_content(values)

    comments = soup.find('ul', {'class': 'fdb_lst_ul '})
    comments = comments.find_all('li')
    for comment in comments:
        vote = comment.find('span', {'class': 'voted_count'}).text.strip()
        if not vote:  # 빈값이면
            vote = 0
        comment_content = comment.find('div', {'class': 'xe_content'})
        cc = comment_content.text.strip()
        if comment_content.find('a'):
            cc = ' '.join(cc.split()[1:])
        values = (vote, cc, article_id)
        commit_comment(values)
    time.sleep(10)



  0%|          | 0/644 [00:00<?, ?it/s]

  0%|          | 2/644 [00:00<00:44, 14.43it/s]

  1%|          | 4/644 [00:00<00:41, 15.32it/s]

  1%|          | 6/644 [00:00<00:39, 16.30it/s]

  1%|▏         | 9/644 [00:00<00:35, 17.85it/s]

  2%|▏         | 11/644 [00:00<00:35, 17.93it/s]

  2%|▏         | 11/644 [00:15<00:35, 17.93it/s]

  2%|▏         | 13/644 [00:15<24:18,  2.31s/it]

  2%|▏         | 14/644 [00:33<1:13:22,  6.99s/it]

  2%|▏         | 15/644 [00:56<2:02:06, 11.65s/it]

  2%|▏         | 16/644 [01:08<2:04:50, 11.93s/it]

  3%|▎         | 17/644 [01:24<2:16:40, 13.08s/it]

  3%|▎         | 18/644 [01:39<2:22:08, 13.62s/it]

  3%|▎         | 19/644 [01:51<2:17:35, 13.21s/it]

  3%|▎         | 20/644 [02:07<2:25:58, 14.04s/it]

  3%|▎         | 21/644 [02:24<2:34:34, 14.89s/it]

  3%|▎         | 22/644 [02:36<2:24:56, 13.98s/it]

  4%|▎         | 23/644 [02:50<2:24:44, 13.98s/it]

AttributeError: 'NoneType' object has no attribute 'find'

In [160]:
con.close()